# Skyflow Unity Catalog External Functions - Complete Setup & Usage

This notebook creates and demonstrates **persistent** Skyflow external functions in Unity Catalog.

## Quick Start

1. **Configure cell 1** with your credentials
2. **Run cells 1-3** to create the functions
3. **Functions are created** and ready to use immediately!

## Prerequisites

- ✅ Unity Catalog enabled in your workspace
- ✅ CREATE FUNCTION privilege on the target catalog/schema
- ✅ Cluster with Unity Catalog access
- ✅ Lambda API deployed and accessible from Databricks

## Key Differences from Temporary UDF Approach

| Feature | Temporary UDFs | Unity Catalog Functions |
|---------|----------------|-------------------------|
| **Setup** | Run notebook cells | Create functions once |
| **Lifecycle** | Session-scoped | Permanent (cluster-wide) |
| **Availability** | Single session | All users with permission |
| **Configuration** | In notebook | Embedded in functions |
| **Cluster/Vault** | Pass as headers | Hardcoded at creation |
| **Views** | Must be temporary | Can be persistent |
| **Use Case** | Ad-hoc analysis | Production workloads |

## Simplified Function Signatures

Cluster_id and vault_id are **embedded in the function definitions**:

```python
# Temporary UDF approach (databricks_skyflow.ipynb)
skyflow_tokenize(value)  # Cluster/vault in request headers

# Unity Catalog approach (this notebook)
skyflow_tokenize(value, table_name, column_name)  # Cluster/vault embedded
skyflow_detokenize(token)  # Cluster/vault embedded
```

---

# Setup: Create External Functions

Run cells 1-3 to create the functions.

In [ ]:
# ============================================================================
# Step 1: Configuration - UPDATE THESE VALUES
# ============================================================================

# Unity Catalog configuration
CATALOG = "your_catalog_name"
SCHEMA = "your_schema_name"

# Lambda API configuration
LAMBDA_URL = "https://YOUR_API_ID.execute-api.YOUR_REGION.amazonaws.com/processDatabricks"

# Skyflow configuration (embedded in functions)
CLUSTER_ID = "YOUR_CLUSTER_ID"
VAULT_ID = "YOUR_VAULT_ID"

# Skyflow table name for tokenization examples
TABLE = "TABLE_NAME"

# Performance tuning
REQUEST_TIMEOUT = 30  # Timeout in seconds (must be <= Lambda timeout, default 30s)

# Set default catalog and schema
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

print("=" * 60)
print("Configuration Summary")
print("=" * 60)
print(f"Catalog:     {CATALOG}")
print(f"Schema:      {SCHEMA}")
print(f"Lambda URL:  {LAMBDA_URL}")
print(f"Cluster ID:  {CLUSTER_ID}")
print(f"Vault ID:    {VAULT_ID}")
print(f"Table:       {TABLE}")
print(f"Timeout:     {REQUEST_TIMEOUT}s")
print("=" * 60)
print("\n✓ Configuration loaded")
print("\nNext: Run cells 2-3 to create functions")

In [ ]:
# ============================================================================
# Step 2: Create Tokenization External Function
# ============================================================================
#
# NOTE: Unity Catalog SQL UDFs are SCALAR functions that process one row at a time.
# Unlike Pandas UDFs (used in databricks_skyflow.ipynb), these functions call the
# Lambda API for each individual value.
#
# Performance characteristics:
# - Pro: Spark handles parallelism across partitions automatically
# - Pro: Functions are persistent and available cluster-wide
# - Con: More API calls than batched Pandas UDFs (1 call per row vs batched calls)
#
# For high-volume tokenization with optimal batching, use databricks_skyflow.ipynb instead.
#

tokenize_function_sql = f"""
CREATE OR REPLACE FUNCTION skyflow_tokenize(
  value STRING,
  table_name STRING,
  column_name STRING
)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Tokenize a value using Skyflow Lambda API (scalar function)'
AS $$
import requests
import json

def tokenize(value, table_name, column_name):
    if value is None:
        return None

    # Skyflow and Lambda configuration (embedded at function creation time)
    CLUSTER_ID = '{CLUSTER_ID}'
    VAULT_ID = '{VAULT_ID}'
    LAMBDA_URL = '{LAMBDA_URL}'
    REQUEST_TIMEOUT = {REQUEST_TIMEOUT}

    # Prepare request (single value)
    endpoint = LAMBDA_URL
    payload = {{
        "records": [{{column_name: value}}]
    }}

    headers = {{
        "Content-Type": "application/json",
        "X-Skyflow-Operation": "tokenize",
        "X-Skyflow-Cluster-ID": CLUSTER_ID,
        "X-Skyflow-Vault-ID": VAULT_ID,
        "X-Skyflow-Table": table_name
    }}

    # Call Lambda API
    response = requests.post(endpoint, json=payload, headers=headers, timeout=REQUEST_TIMEOUT)
    response.raise_for_status()

    # Extract token from response
    data = response.json().get("data", [])
    if data and len(data) > 0:
        return data[0].get(column_name)

    return None

return tokenize(value, table_name, column_name)
$$
"""

spark.sql(tokenize_function_sql)

print("✓ Created function: skyflow_tokenize(value, table_name, column_name)")
print(f"  Lambda URL: {LAMBDA_URL}")
print(f"  Cluster ID: {CLUSTER_ID}")
print(f"  Vault ID: {VAULT_ID}")
print(f"  Timeout: {REQUEST_TIMEOUT}s")
print("")
print("⚠️  NOTE: This is a scalar function (1 API call per row)")
print("   Spark parallelizes across partitions for performance")
print("   For optimal batching, use databricks_skyflow.ipynb with Pandas UDFs")

In [ ]:
# ============================================================================
# Step 3: Create Detokenization External Function
# ============================================================================
#
# NOTE: This is also a SCALAR function (1 API call per token).
#

detokenize_function_sql = f"""
CREATE OR REPLACE FUNCTION skyflow_detokenize(token STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Detokenize a token using Skyflow Lambda API (scalar function)'
AS $$
import requests
import json

def detokenize(token):
    if token is None:
        return None

    # Skyflow and Lambda configuration (embedded at function creation time)
    CLUSTER_ID = '{CLUSTER_ID}'
    VAULT_ID = '{VAULT_ID}'
    LAMBDA_URL = '{LAMBDA_URL}'
    REQUEST_TIMEOUT = {REQUEST_TIMEOUT}

    # Prepare request (single token)
    endpoint = LAMBDA_URL
    payload = {{
        "tokens": [token]
    }}

    headers = {{
        "Content-Type": "application/json",
        "X-Skyflow-Operation": "detokenize",
        "X-Skyflow-Cluster-ID": CLUSTER_ID,
        "X-Skyflow-Vault-ID": VAULT_ID
    }}

    # Call Lambda API
    response = requests.post(endpoint, json=payload, headers=headers, timeout=REQUEST_TIMEOUT)
    response.raise_for_status()

    # Extract value from response
    data = response.json().get("data", [])
    if data and len(data) > 0:
        return data[0].get("value")

    return None

return detokenize(token)
$$
"""

spark.sql(detokenize_function_sql)

print("✓ Created function: skyflow_detokenize(token)")
print(f"  Lambda URL: {LAMBDA_URL}")
print(f"  Cluster ID: {CLUSTER_ID}")
print(f"  Vault ID: {VAULT_ID}")
print(f"  Timeout: {REQUEST_TIMEOUT}s")

## Setup Complete!

The following persistent functions have been created in Unity Catalog:

1. **skyflow_tokenize(value, table_name, column_name)** - Tokenizes sensitive data
2. **skyflow_detokenize(token)** - Detokenizes tokens back to plaintext

These functions are now:
- Available to all users with EXECUTE permission
- Persistent across cluster restarts
- Configured with your Cluster ID, Vault ID, and Lambda URL

**Optional:** Grant permissions to other users:
```sql
GRANT EXECUTE ON FUNCTION skyflow_tokenize TO `data_engineers`;
GRANT EXECUTE ON FUNCTION skyflow_detokenize TO `data_engineers`;
```

---

# Usage Examples

The cells below demonstrate how to use the functions.

## Verify Functions Exist

Check that the external functions were created successfully:

In [ ]:
# List all functions in the schema
functions_df = spark.sql(f"""
    SHOW FUNCTIONS IN {CATALOG}.{SCHEMA}
    LIKE 'skyflow*'
""")

display(functions_df)

# Verify both functions exist
function_names = [row.function for row in functions_df.collect()]
expected = [
    f"{CATALOG}.{SCHEMA}.skyflow_tokenize",
    f"{CATALOG}.{SCHEMA}.skyflow_detokenize"
]

for func in expected:
    if func in function_names:
        print(f"✓ Found: {func}")
    else:
        print(f"✗ Missing: {func}")
        print(f"  Re-run cells 2-3 to create functions")

## Generate Test Data

Create sample data for testing the functions:

In [ ]:
from pyspark.sql.functions import expr, current_timestamp

# Configure number of test rows
NUM_ROWS = 100

# Create test data
test_df = spark.range(NUM_ROWS).select(
    (expr("id + 1").alias("user_id")),
    expr("concat('user_', id)").alias("username"),
    expr("concat('user', id, '@example.com')").alias("email"),
    expr("concat('555-01-', LPAD(id % 10000, 4, '0'))").alias("ssn"),
    expr("concat('+1-555-', LPAD(id % 1000, 3, '0'), '-', LPAD((id * 7) % 10000, 4, '0'))").alias("phone"),
    current_timestamp().alias("created_at")
)

# Save as table
test_df.write.mode("overwrite").saveAsTable("raw_users")

print(f"✓ Created raw_users table with {NUM_ROWS} rows")
display(spark.table("raw_users").limit(10))

## Tokenize Single Column

Test the tokenization function on sample data:

In [ ]:
# Tokenize emails using the persistent external function
tokenized_df = spark.sql(f"""
    SELECT
        user_id,
        username,
        email,
        skyflow_tokenize(email, '{TABLE}', 'email') as email_token,
        phone
    FROM raw_users
    LIMIT 10
""")

display(tokenized_df)

## Create Persistent Tokenized Table

Unlike temporary UDFs, Unity Catalog functions can create **permanent tables**:

In [ ]:
# Create persistent tokenized table
spark.sql(f"""
    CREATE OR REPLACE TABLE tokenized_users AS
    SELECT
        user_id,
        username,
        skyflow_tokenize(email, '{TABLE}', 'email') as email_token,
        phone,
        created_at
    FROM raw_users
""")

count = spark.table("tokenized_users").count()
print(f"✓ Created tokenized_users table with {count} rows")
display(spark.table("tokenized_users").limit(10))

## Detokenize Tokens

Test the detokenization function:

In [ ]:
# Detokenize tokens
detokenized_df = spark.sql("""
    SELECT
        user_id,
        username,
        email_token,
        skyflow_detokenize(email_token) as email_detokenized,
        phone
    FROM tokenized_users
    LIMIT 10
""")

display(detokenized_df)
print("\n✓ Roundtrip complete: Tokens → Detokenized values")

## Create Persistent Detokenized View

**This is the key advantage**: Unity Catalog functions allow **persistent views**, unlike temporary UDFs:

In [ ]:
# Create PERSISTENT view (not possible with temporary UDFs!)
spark.sql("""
    CREATE OR REPLACE VIEW users_detokenized AS
    SELECT
        user_id,
        username,
        skyflow_detokenize(email_token) as email,
        phone,
        created_at
    FROM tokenized_users
""")

print("✓ Created PERSISTENT view: users_detokenized")
print("  This view is available to all users and survives cluster restarts!")
display(spark.sql("SELECT * FROM users_detokenized LIMIT 10"))

## Verify Roundtrip Accuracy

Compare original values with tokenized and detokenized values:

In [ ]:
# Compare original vs detokenized
verification_df = spark.sql("""
    SELECT
        t.user_id,
        r.email as original_email,
        t.email_token,
        d.email as detokenized_email,
        CASE
            WHEN r.email = d.email THEN 'MATCH'
            ELSE 'MISMATCH'
        END as verification
    FROM tokenized_users t
    JOIN raw_users r ON t.user_id = r.user_id
    JOIN users_detokenized d ON t.user_id = d.user_id
    LIMIT 10
""")

display(verification_df)

# Check for any mismatches
mismatches = verification_df.filter("verification = 'MISMATCH'").count()
if mismatches == 0:
    print("\n✓ All records match! Tokenization → Detokenization working correctly.")
else:
    print(f"\n✗ Found {mismatches} mismatches - investigate!")

## Cleanup (Optional)

To remove all test resources:

In [ ]:
# Uncomment to drop test tables and views
# spark.sql("DROP TABLE IF EXISTS raw_users")
# spark.sql("DROP TABLE IF EXISTS tokenized_users")
# spark.sql("DROP VIEW IF EXISTS users_detokenized")
# spark.sql("DROP FUNCTION IF EXISTS skyflow_detokenize")
# spark.sql("DROP FUNCTION IF EXISTS skyflow_tokenize")
# print("✓ Cleanup complete")

## Summary

This notebook demonstrated Unity Catalog external functions for Skyflow integration:

**Key Features:**
- Simple setup - Create functions directly from this notebook (cells 1-3)  
- No connection object needed - Lambda URL embedded directly in functions
- Persistent functions - Available across all sessions and users  
- Persistent views - Create permanent detokenized views  
- Production-ready - Suitable for ETL pipelines and production workloads  
- Access control - Leverage Unity Catalog permissions + Skyflow governance  
- Embedded configuration - Cluster ID, Vault ID, and Lambda URL hardcoded in functions

**Important Performance Note:**

Unity Catalog SQL UDFs are **scalar functions** that process one row at a time:
- Makes 1 API call per row (not batched)
- Spark parallelizes across partitions for performance
- Suitable for moderate-volume workloads where persistence is more important than API call optimization

**When to use Unity Catalog vs Temporary UDFs:**

| Use Case | Recommended Approach |
|----------|---------------------|
| Production pipelines, shared resources | Unity Catalog (this notebook) |
| High-volume tokenization (1M+ rows) | Temporary Pandas UDFs (databricks_skyflow.ipynb) |
| Ad-hoc analysis, development | Temporary Pandas UDFs (databricks_skyflow.ipynb) |
| Persistent views required | Unity Catalog (this notebook) |
| Optimal API call batching required | Temporary Pandas UDFs (databricks_skyflow.ipynb) |

**To update configuration:**
1. Update variables in cell 1
2. Re-run cells 2-3 to recreate functions with new configuration